In [1]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay, reset_replay

enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

versions()

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay

Versions:
- jupyter_cadquery  3.5.2
- cad_viewer_widget 1.4.1
- open cascade      7.6.3

Plugins loaded:
- cadquery-massembly


In [2]:
set_defaults(axes=True, timeit=False)

cv = open_viewer("Unit Test Files", cad_width=640, height=480, glass=True)


enable_replay(False, False)
show_object = replay


Enabling jupyter_cadquery replay


In [3]:
# These can be modified rather than hardcoding values for each dimension.
length = 120.0  # Length of the block
width = 40.0  # Width of the block
thickness = 80.0  # Thickness of the block

feature_b_length = 120.0  # Length of the block's flange
feature_b_width = 40.0  # Width of the block's flange
feature_b_thickness = 40.0  # Thickness of the block's flange

result = (
    cq.Workplane('XZ')
    .box(length, width, thickness)
)

# Displays the result of this script
show_object(result, show_result=False)

Use the multi select box below to select one or more steps you want to examine


* [ ] Change flange to feature name

In [4]:
class CentredLShapedBlock:
    '''
    Represents an L-shaped block constructed from a main block and a flange.

    This class encapsulates the creation of an L-shaped block model using CadQuery. 
    The model consists of a primary rectangular block and a flange extending from one side,
    forming an L-shape. The dimensions of both the main block and the flange can be 
    customized.

    Attributes:
        length (float): The length of the main block.
        width (float): The width of the main block.
        thickness (float): The thickness (height) of the main block.
        feature_b_length (float): The length of the flange.
        feature_b_width (float): The width of the flange, matching the width of the main block.
        feature_b_thickness (float): The thickness (height) of the flange.
        model (cadquery.Workplane): The resulting CadQuery Workplane object after combining
                                    the main block and the flange.

    Methods:
        _create_model: Internal method to construct the L-shaped block.
        show: Displays the constructed model in a CadQuery viewer.

    Example:
        >>> block = LShapedBlock(120.0, 80.0, 40.0, 40.0, 80.0, 20)
        >>> block.show()

    Note:
        The L-shaped block is constructed with the main block centered on the 'XZ' workplane,
        and the flange positioned relative to it to form the L-shape.

    TODO Change widths to be the same
    TODO Change thickness and width
    '''
    def __init__(self, length, width, thickness, feature_b_length, feature_b_width, feature_b_thickness):
        '''
        Initializes the LShapedBlock with specified dimensions for the main block and flange.

        Parameters:
            length (float): The length of the main block.
            width (float): The width of the main block.
            thickness (float): The thickness (height) of the main block.
            feature_b_length (float): The length of the flange.
            feature_b_width (float): The width of the flange, matching the width of the main block.
            feature_b_thickness (float): The thickness (height) of the flange.
        '''
        self.length = length
        self.width = width
        self.thickness = thickness
        self.feature_b_length = feature_b_length
        self.feature_b_width = feature_b_width
        self.feature_b_thickness = feature_b_thickness
        self.model = self._create_model()
        

    def _create_model(self):
        '''
        Constructs the L-shaped block from the main block and flange.
        '''
        # Create the main block
        block = cq.Workplane('XZ').box(self.length, self.thickness, self.width)

        # Create the flange
        flange = cq.Workplane('XZ', origin=(
            self.feature_b_length/2 - self.length/2,
            0,
            self.thickness/2 + self.feature_b_thickness/2
        )).box(self.feature_b_length, self.feature_b_thickness, self.feature_b_width)

        # Combine the block and the flange to form an L-shaped profile
        return block.union(flange)
    

    def show(self):
        '''
        Displays the constructed L-shaped block model in a CadQuery viewer.
        '''
        show_object(self.model)

In [5]:
# Usage
block = CentredLShapedBlock(120.0, 80.0, 40.0, 40.0, 80.0, 20)
block.show()

Use the multi select box below to select one or more steps you want to examine


## Models
- [ ] Why using Workplane instead of Solid?

In [6]:
class Model:
    """
    A generic model class that serves as a base for different types of CAD models.
    
    Attributes:
        length (float): The length of the model.
        width (float): The width of the model.
        height (float): The height of the model.
        model (cadquery.Workplane): The resulting CadQuery Workplane object representing the model.
    """
    def __init__(self, length, width, thickness, model=None):
        """
        Initializes the Model with specified dimensions.
        
        Parameters:
            length (float): The length of the model.
            width (float): The width of the model.
            height (float): The height of the model.
            model (cadquery.Workplane, optional): An existing CadQuery Workplane object to use as the model.
        """
        if any(d <= 0 for d in [length, width, thickness]):
            print(length)
            print(width)
            print(thickness)            
            raise ValueError("All dimensions must be positive numbers. Model")
        
        self.length = length
        self.width = width
        self.thickness = thickness
        if model is None:
            self.model = self._create_model()
        else:
            self.model = model
            

    def _create_model(self):
        """
        Creates a basic box model.
        """
        return cq.Workplane("XY").box(self.length, self.width, self.thickness)
    
    
    def cut(self, other):
        """
        Performs a cut operation between this model and another model.

        Parameters:
            other (Model): The model to subtract from this model.

        Returns:
            Model: A new Model instance representing the result of the cut operation.
        """
        try:
            return Model._cq_cut(self, other)
        except Exception as e:
            print(f"Error during subtraction: {e}")
    

    @classmethod
    def _cq_cut(cls, this_model, other_model):
        """
        Performs the actual CadQuery cut operation between two models and calculates the new dimensions based on the resulting model's bounding box.

        Parameters:
            this_model (Model):  The model from which to subtract.
            other_model (Model): The model to subtract.

        Returns:
            Model or None: A new Model instance representing the result of the cut operation, with dimensions based on the resulting bounding box, or None if the operation is not successful.
        """
        try:
            result_model = this_model.model.cut(other_model.model)
            
            # Calculate the bounding box dimensions of the resulting model
            BB =  result_model.val().BoundingBox()


            # Calculate length, width, and thickness based on bounding box
            length = BB.xlen
            width = BB.ylen
            thickness = BB.zlen
            
            # Return a new Model instance with calculated dimensions and the resulting CadQuery workplane object
            return cls(length, width, thickness, model=result_model)
        except Exception as e:
            print(f"Unexpected error during cut operation: {e}")
            return None


    def show(self):
        """
        Displays the constructed model in a CadQuery viewer.
        """
        if self.model:
            show_object(self.model)
        else:
            print("Model not created yet.")


### Block Example

In [7]:
class LShapedBlock(Model):
    """
    Represents an L-shaped block constructed from a main block and a feature (referred to as feature_b).

    This class encapsulates the creation of an L-shaped block model using CadQuery. 
    The model consists of a primary rectangular block and feature_b extending from one side,
    forming an L-shape. The dimensions of both the main block and feature_b can be 
    customized.

    Attributes:
        length (float): The length of the main block.
        width (float): The width of the main block.
        thickness (float): The thickness (height) of the main block.
        feature_b_length (float): The length of feature_b.
        feature_b_width (float): The width of feature_b, matching the width of the main block.
        feature_b_thickness (float): The thickness (height) of feature_b.
        model (cadquery.Workplane): The resulting CadQuery Workplane object after combining
                                    the main block and feature_b.

    Methods:
        _create_model: Internal method to construct the L-shaped block.

    Example:
        >>> block = LShapedBlock(120.0, 80.0, 40.0, 40.0, 80.0, 20)
        >>> block.show()

    Note:
        The L-shaped block is constructed with the main block centered on the 'XZ' workplane,
        and feature_b positioned relative to it to form the L-shape.
    """
    def __init__(self, length, width, thickness, feature_b_length, feature_b_width, feature_b_thickness):
        """
        Initializes the LShapedBlock with specified dimensions for the main block and feature_b.

        Parameters:
            length (float): The length of the main block.
            width (float): The width of the main block.
            thickness (float): The thickness (height) of the main block.
            feature_b_length (float): The length of feature_b.
            feature_b_width (float): The width of feature_b, matching the width of the main block.
            feature_b_thickness (float): The thickness (height) of feature_b.
        """
        # Check for positive values
        if any(d <= 0 for d in [length, width, thickness, feature_b_length, 
                                feature_b_width, feature_b_thickness]):
            raise ValueError("All dimensions must be positive numbers.")
            
        self.length = length
        self.width = width
        self.thickness = thickness
        self.feature_b_length = feature_b_length
        self.feature_b_width = feature_b_width
        self.feature_b_thickness = feature_b_thickness
        self.model = self._create_model()

    def _create_model(self):
        """
        Constructs the L-shaped block from the main block and feature_b.
        """
        # Create the main block
        block = (
            cq.Workplane("XY")
            .box(self.length, self.width, self.thickness, centered=(False, False, False))
            # .translate((-self.length, -self.width, 0))  # Shift to position lower right at origin
        )

        # Create feature_b
        feature_b = (
            cq.Workplane("XY")
            .box(self.feature_b_length, self.feature_b_width, self.feature_b_thickness, centered=(False, False, False))
            # .translate((-self.feature_b_length, 0, 0))  # Shift to position correctly relative to the main block
        )

        # Combine the block and feature_b to form an L-shaped profile
        # Assuming feature_b extends outward from the side of the main block
        return block.union(feature_b.translate((0, 0, self.thickness)))

In [8]:
set_defaults(axes=True, timeit=False)

cv = open_viewer("Model A", cad_width=640, height=480, glass=True)


enable_replay(False, False)
show_object = replay

# Example usage:
lshapedblock = LShapedBlock(120.0, 80.0, 40.0, 40.0, 80.0, 20.0)
lshapedblock.show()


Enabling jupyter_cadquery replay
Use the multi select box below to select one or more steps you want to examine


### Extrusion Example
LShapedExtrude creates a model from an extrusion of a more complex sketch describing the full profile of the model.

In [9]:
class LShapedExtrude(Model):
    """
    Represents an L-shaped block created by extruding an L-shaped sketch.

    This class encapsulates the creation of an L-shaped block model using CadQuery by
    sketching an L-shape and then extruding it. The dimensions of the L-shape can be
    customized, including the lengths of its sides, the width, and the extrusion thickness.

    Attributes:
        length (float): The horizontal length of the L shape.
        width (float): The vertical length of the L shape.
        thickness (float): The thickness of the extrusion.
        side_width (float): The height difference between the two vertical sides.
        side_thickness (float): The width of the sides of the L shape.
    
        model (cadquery.Workplane): The resulting CadQuery Workplane object after extruding
                                    the L-shaped sketch.

    Methods:
        _create_sketch: Internal method to create the L-shaped sketch.
        _create_model: Internal method to extrude the sketch and form the solid.

    Example:
        >>> l_block = LShapedExtrude(120.0, 80.0, 40.0, 40.0, 20.0)
        >>> l_block.show()
    """
    def __init__(self, length, width, thickness, side_width, side_thickness):
        """
        Initializes the LShapedExtrude with specified dimensions for the L-shaped sketch and
        extrusion.

        Parameters:
            length (float): The horizontal length of the L shape.
            width (float): The vertical length of the L shape.
            thickness (float): The thickness of the extrusion.
            side_width (float): The height difference between the two vertical sides.
            side_thickness (float): The width of the sides of the L shape.
        """
        # Check for positive values
        if any(d <= 0 for d in [length, width, thickness, side_width, side_thickness]):
            raise ValueError("All dimensions must be positive numbers.")
            
        self.length = length
        self.width = width
        self.thickness = thickness
        self.side_width = side_width
        self.side_thickness = side_thickness
        self.model = self._create_model()
        
    def _create_sketch(self):
        """
        Creates the L-shaped sketch using provided parameters.
        """
        l_shape_sketch = (
            cq.Workplane("XZ")
            .moveTo(0, 0)  # Starting point at the origin
            .vLineTo(self.thickness+self.side_thickness)  # Vertical line up
            .hLineTo(self.side_width)  # Horizontal line to the right
            .vLineTo(self.thickness)  # Vertical line down
            .hLineTo(self.length)  # Horizontal line to the left
            .vLineTo(0)  # Close the sketch vertically
            .hLineTo(0)  # Close the sketch horizontally
            .close()  # Ensure the sketch forms a complete L shape
        )
        return l_shape_sketch


    def _create_model(self):
        """
        Creates the L-shaped model by sketching and extruding the L-shape.
        """
        
        l_shape_sketch = self._create_sketch()

        # Extrude the L-shaped sketch to create a solid
        return l_shape_sketch.extrude(-self.width)


In [10]:
set_defaults(axes=True, timeit=False)

cv = open_viewer("Model B", cad_width=640, height=480, glass=True)


enable_replay(False, False)
show_object = replay

# Example usage:
lshapedextrude = LShapedExtrude(120.0, 80.0, 40.0, 40.0, 20.0)
lshapedextrude.show()


Enabling jupyter_cadquery replay
Use the multi select box below to select one or more steps you want to examine


In [11]:
class RefactoringTest:
    """
    Tests the equivalence of CAD models by attempting to subtract each model from the other.
    
    This class leverages the `subtract` method of the `Model` class to determine if two models are
    geometrically and locationally identical. If subtracting one model from another results in an 
    operation error indicative of a void model, it suggests the models are identical.
    
    Attributes:
        reference_model (Model): The original model against which refactored models are compared.
    """
    
    def __init__(self, reference_model):
        """
        Initializes the RefactoringTest with a reference model.
        
        Parameters:
            reference_model (Model): The reference model for comparison.
        """
        self.reference_model = reference_model

    def run(self, refactored_model):
        """
        Runs the refactoring test by comparing the reference model with a refactored model.
        
        Parameters:
            refactored_model (Model): The refactored model to compare against the reference model.
        
        Returns:
            bool: False if both subtraction operations indicate the models are identical (i.e., both
                  operations fail due to producing a void model), True otherwise.
        """
        # Attempt to subtract the refactored model from the reference model
        subtract_result_a = self.reference_model.cut(refactored_model)
        # Attempt to subtract the reference model from the refactored model
        subtract_result_b = refactored_model.cut(self.reference_model)
        
        # If both subtraction operations were successful, it implies there are differences
        if not subtract_result_a and not subtract_result_b:
            return True  # Models are not identical
        else:
            # If at least one subtraction fails, it suggests potential identity,
            # but more checks might be needed depending on the specifics of subtraction failures.
            return False


In [12]:
RT = RefactoringTest(lshapedblock)

In [13]:
RT.run(lshapedextrude)

Unexpected error during cut operation: Bnd_Box is void
Unexpected error during cut operation: Bnd_Box is void


True

In [14]:
A = LShapedExtrude(100.0, 80.0, 40.0, 40.0, 20.0)

In [15]:
RT.run(A)

Unexpected error during cut operation: Bnd_Box is void


False